# Equations of motion for control moment gyroscope

In [1]:
import sympy as sym
import numpy as np

The `sympy.physics.mechanics` module makes it easier to work with (and display) time derivatives.

In [2]:
from sympy.physics import mechanics
mechanics.init_vprinting()

Define parameters in symbolic form.

In [3]:
# Principal moments of inertia
J_1x, J_1y, J_1z = sym.symbols('J_1x, J_1y, J_1z')
J_2x, J_2y, J_2z = sym.symbols('J_2x, J_2y, J_2z')
J_3x, J_3y, J_3z = sym.symbols('J_3x, J_3y, J_3z')

# Moment of inertia matrices in body-fixed reference frames
J1 = sym.Matrix.diag(J_1x, J_1y, J_1z)
J2 = sym.Matrix.diag(J_2x, J_2y, J_2z)
J3 = sym.Matrix.diag(J_3x, J_3y, J_3z)

# Spar length
r = sym.symbols('r')

# Load mass
m = sym.symbols('m')

# Acceleration of gravity
g = sym.symbols('g')

Define variables.

In [4]:
# Time
t = sym.Symbol('t')

# Joint angles:
q1, q2, q3 = mechanics.dynamicsymbols('q1, q2, q3')

# Joint velocities
v1 = q1.diff(t)
v2 = q2.diff(t)
v3 = q3.diff(t)

# Joint accelerations
a1 = v1.diff(t)
a2 = v2.diff(t)
a3 = v3.diff(t)

# Torques:
tau2, tau3 = sym.symbols('tau2, tau3')

Compute angular velocity of each link with respect to body-fixed reference frames (the choice of frame is very important - using a space-fixed reference frame is a common mistake).

In [5]:
c1 = sym.cos(q1)
s1 = sym.sin(q1)
R_p_in_w = sym.Matrix([[c1, -s1, 0], [s1, c1, 0], [0, 0, 1]])

c2 = sym.cos(q2)
s2 = sym.sin(q2)
R_g_in_p = sym.Matrix([[1, 0, 0], [0, c2, -s2], [0, s2, c2]])

w1 = sym.Matrix([[0], [0], [v1]])
w2 = R_p_in_w.T * w1 + sym.Matrix([[v2], [0], [0]])
w3 = R_g_in_p.T * w2 + sym.Matrix([[0], [-v3], [0]])

# Show result
w1, w2, w3

C:\Users\david\anaconda3\lib\site-packages\IPython\lib\latextools.py:126: MatplotlibDeprecationWarning: 
The to_png function was deprecated in Matplotlib 3.4 and will be removed two minor releases later. Use mathtext.math_to_image instead.
  mt.to_png(f, s, fontsize=12, dpi=dpi, color=color)
C:\Users\david\anaconda3\lib\site-packages\IPython\lib\latextools.py:126: MatplotlibDeprecationWarning: 
The to_rgba function was deprecated in Matplotlib 3.4 and will be removed two minor releases later. Use mathtext.math_to_image instead.
  mt.to_png(f, s, fontsize=12, dpi=dpi, color=color)
C:\Users\david\anaconda3\lib\site-packages\IPython\lib\latextools.py:126: MatplotlibDeprecationWarning: 
The to_mask function was deprecated in Matplotlib 3.4 and will be removed two minor releases later. Use mathtext.math_to_image instead.
  mt.to_png(f, s, fontsize=12, dpi=dpi, color=color)
C:\Users\david\anaconda3\lib\site-packages\IPython\lib\latextools.py:126: MatplotlibDeprecationWarning: 
The MathtextBa

⎛⎡0 ⎤  ⎡q₂̇⎤  ⎡      q₂̇       ⎤⎞
⎜⎢  ⎥  ⎢  ⎥  ⎢               ⎥⎟
⎜⎢0 ⎥, ⎢0 ⎥, ⎢sin(q₂)⋅q₁̇ - q₃̇⎥⎟
⎜⎢  ⎥  ⎢  ⎥  ⎢               ⎥⎟
⎝⎣q₁̇⎦  ⎣q₁̇⎦  ⎣  cos(q₂)⋅q₁̇   ⎦⎠

Compute position and velocity of load mass.

In [6]:
p = R_p_in_w * sym.Matrix([-r, 0, 0])
v = p.diff(t)

# Show result
p, v

⎛⎡-r⋅cos(q₁)⎤  ⎡r⋅sin(q₁)⋅q₁̇ ⎤⎞
⎜⎢          ⎥  ⎢             ⎥⎟
⎜⎢-r⋅sin(q₁)⎥, ⎢-r⋅cos(q₁)⋅q₁̇⎥⎟
⎜⎢          ⎥  ⎢             ⎥⎟
⎝⎣    0     ⎦  ⎣      0      ⎦⎠

Compute Lagrangian.

In [7]:
T = ((w1.T * J1 * w1) + (w2.T * J2 * w2) + (w3.T * J3 * w3) + (v.T * m * v)) / 2
V = m * g * (sym.Matrix([1, 0, 0]).T * p)
L = sym.simplify(T - V)

# Show result
L

⎡       2         2          2         2           2       2                  
⎢J_1z⋅q₁̇    J₂ₓ⋅q₂̇    J_2z⋅q₁̇    J₃ₓ⋅q₂̇    J_3y⋅sin (q₂)⋅q₁̇              
⎢──────── + ─────── + ──────── + ─────── + ───────────────── - J_3y⋅sin(q₂)⋅q₁
⎣   2          2         2          2              2                          

             2           2       2          2                      2   2⎤
           J_3y⋅q₃̇    J_3z⋅sin (q₂)⋅q₁̇    J_3z⋅q₁̇                    m⋅r ⋅q
̇⋅q₃̇ + ──────── - ───────────────── + ──────── + g⋅m⋅r⋅cos(q₁) + ────────⎥
         2               2              2                          2    ⎦

Compute equations of motion.

In [8]:
EOM = L.jacobian([v1, v2, v3]).diff(t) - L.jacobian([q1, q2, q3]) - sym.Matrix([0, tau2, tau3]).T

Simplify equations of motion by solving for accelerations.

In [9]:
sol = sym.solve(EOM, [a1, a2, a3])
h = sym.together(sym.simplify(sym.Matrix([sol[a1], sol[a2], sol[a3]]), full=True))

Assume $\tau_3=0$ and look only at the equations of motion that govern the platform and the gimbal:

In [10]:
h = h[0:2, 0].subs(tau3, 0)

Display the vector-valued function $h$ for which the equations of motion can be written as

$$\begin{bmatrix} \ddot{q}_1 \\ \ddot{q}_2 \end{bmatrix} = h(q_1, q_2, \dot{q}_1, \dot{q}_2, \tau_2)$$

where everything else (mass and inertial parameters, spar length, acceleration of gravity, and rotor velocity $\dot{q}_3$) is assumed constant.

In [11]:
h

⎡-J_3y⋅sin(2⋅q₂)⋅q₁̇⋅q₂̇ + 2⋅J_3y⋅cos(q₂)⋅q₂̇⋅q₃̇ + 2⋅J_3z⋅sin(2⋅q₂)⋅q₁̇⋅q₂̇ -
⎢─────────────────────────────────────────────────────────────────────────────
⎢                           ⎛                      2          2⎞              
⎢                         2⋅⎝J_1z + J_2z + J_3z⋅cos (q₂) + m⋅r ⎠              
⎢                                                                             
⎢                          2                                           2      
⎢         J_3y⋅sin(2⋅q₂)⋅q₁̇  - 2⋅J_3y⋅cos(q₂)⋅q₁̇⋅q₃̇ - J_3z⋅sin(2⋅q₂)⋅q₁̇  +
⎢         ────────────────────────────────────────────────────────────────────
⎣                                     2⋅(J₂ₓ + J₃ₓ)                           

 2⋅g⋅m⋅r⋅sin(q₁)⎤
──────────⎥
          ⎥
          ⎥
          ⎥
          ⎥
 2⋅τ₂         ⎥
─         ⎥
          ⎦

Display $h$ as latex-formatted text that could be copy/pasted into a $\LaTeX$ document (or into markdown).

In [12]:
h=h.subs(J_1z, 0.5)
h=h.subs(J_2x, 0.001)
h=h.subs(J_2z, 0.001)
h=h.subs(J_3x, 0.01)
h=h.subs(J_3y, 0.01)
h=h.subs(J_3y, 0.01)
h=h.subs(J_3z, 0.01)
h=h.subs(m, 1.0)
h=h.subs(r, 2.0)
h=h.subs(g, 9.81)

In [13]:
h

⎡-39.24⋅sin(q₁) + 0.01⋅sin(2⋅q₂)⋅q₁̇⋅q₂̇ + 0.02⋅cos(q₂)⋅q₂̇⋅q₃̇⎤
⎢──────────────────────────────────────────────────────────⎥
⎢                  ⎛        2            ⎞                 ⎥
⎢                2⋅⎝0.01⋅cos (q₂) + 4.501⎠                 ⎥
⎢                                                          ⎥
⎣  90.9090909090909⋅τ₂ - 0.909090909090909⋅cos(q₂)⋅q₁̇⋅q₃̇   ⎦

In [14]:
h[0]
mechanics.mprint(h[0])
mechanics.mprint(h[1])

(-39.24*sin(q1) + 0.01*sin(2*q2)*q1'*q2' + 0.02*cos(q2)*q2'*q3')/(2*(0.01*cos(q2)**2 + 4.501))
90.9090909090909*tau2 - 0.909090909090909*cos(q2)*q1'*q3'


In [17]:
v3 = 1000
q1,q2,v1,v2,tau = sym.symbols('q1,q2,v1,v2,tau')
f = sym.Matrix([v1,v2,((-39.24*sym.sin(q1) + 0.01*sym.sin(2*q2)*v1*v2 + 0.02*sym.cos(q2)*v2*v3)/(2*(0.01*sym.cos(q2)**2 + 4.501))),(90.9090909090909*tau - 0.909090909090909*sym.cos(q2)*v1*v3)])

q1_e = 2*np.pi
q2_e = 0.
v1_e = 0.
v2_e = 0.
tau_e = 0.

f1_num = sym.lambdify([q1,q2,v1,v2,tau],f)
f1_num(q1_e, q2_e, v1_e, v2_e, tau_e)
A_num = sym.lambdify((q1,q2,v1,v2,tau), f.jacobian([q1,q2,v1,v2]))
A=A_num(q1_e, q2_e, v1_e, v2_e, tau_e)

B_num = sym.lambdify([q1,q2,v1,v2,tau], f.jacobian([tau]),'numpy')
B=B_num(q1_e, q2_e, v1_e, v2_e, tau_e)
print(A)
print(B)
f

[[   0.            0.            1.            0.        ]
 [   0.            0.            0.            1.        ]
 [  -4.34936821    0.            0.            2.21680337]
 [   0.            0.         -909.09090909    0.        ]]
[[ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [90.90909091]]


⎡                          v₁                          ⎤
⎢                                                      ⎥
⎢                          v₂                          ⎥
⎢                                                      ⎥
⎢0.01⋅v₁⋅v₂⋅sin(2⋅q₂) + 20.0⋅v₂⋅cos(q₂) - 39.24⋅sin(q₁)⎥
⎢──────────────────────────────────────────────────────⎥
⎢                        2                             ⎥
⎢                0.02⋅cos (q₂) + 9.002                 ⎥
⎢                                                      ⎥
⎣   90.9090909090909⋅τ - 909.090909090909⋅v₁⋅cos(q₂)   ⎦

In [16]:
print(f'{mechanics.mlatex(w1)}')

\left[\begin{matrix}0\\0\\\dot{q}_{1}\end{matrix}\right]
